In [ ]:
!rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/master
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset-tf-2-1-0==1.2.0 

In [2]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

!mkdir -p data/training
!gcsfuse --only-dir training/ waymo_open_dataset_v_1_2_0_individual_files data/training/

!mkdir -p data/testing
!gcsfuse --only-dir testing/ waymo_open_dataset_v_1_2_0_individual_files data/testing/

In [1]:
import os
import gc
import tensorflow as tf
import math
import numpy as np
import itertools
import matplotlib.pyplot as plt
from google.colab import files
import time

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data_utils
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

from Net_Lib import *

list_train = os.listdir(path='data/training')
list_test = os.listdir(path='data/testing')


# LSTM Training

## Batch-Maker

In [2]:
# Data extraction and preparation
def create_LSTMbatch(index,train):
  if(train):
    loc = "data/training/"
    req_list = list_train
  else:
    loc = "data/testing/"
    req_list = list_test

  train_images = []
  train_labels = []
  img_yes = False
  imagenum = 0
  for dataset in list_train[index:index+1]:
    dataset = tf.data.TFRecordDataset(loc+dataset, compression_type='')
    for data in dataset:
      frame = open_dataset.Frame()
      frame.ParseFromString(bytearray(data.numpy()))

      # Get image itself
      for index, image in enumerate(frame.images):
        if(image.name == 1):
          img = tf.image.decode_jpeg(image.image)
          img = tf.image.resize(img, [640, 960])
          train_images.append(img)

      # Get label data
      for cam_labels in frame.projected_lidar_labels:
        if(cam_labels.name != 1):
          continue
        for label in cam_labels.labels:
          train_labels.append(np.array([label.metadata.speed_x, label.metadata.speed_y, label.metadata.accel_x,
                                            label.metadata.accel_y, imagenum]))
          img_yes = True
      
      # Write specialized "empty image" output
      if(not img_yes):
        train_labels.append(np.array([-1,-1,-1,-1,imagenum]))
      else:
        img_yes = False
      imagenum += 1
 
  train_labels = torch.from_numpy(np.array(train_labels))
  train_images = torch.from_numpy(np.array(train_images).transpose(0,3,1,2))
  return train_labels, train_images

## Small-Scale Train

In [ ]:
anchors = torch.load("anch.pt")
model_save_name = 'save_checkpoint.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
checkpoint = torch.load(path)

n_tot_train = len(list_train)  # number of overall training examples

nepoch_readbatch = 25          # number of epochs through training set
batchsize = 16                 # minibatch size
a_cuda = True                  # whether or not to enable cuda gpu acceleration

res = 32                       # resolution of image breakdown
H = 1280//(2*res)                   # height of the grid over images
W = 1920//(2*res)                   # width of the grid over images

network_spec = parse_cfg("yolo.cfg")
module_list = create_network(network_spec, 3)
yolo = Model(module_list).float()
yolo.load_state_dict(checkpoint['model_state_dict'])
recurrent = LSTM_module(W*H*50, 360, 2, W*H*5)

if(a_cuda):
  yolo.cuda()
  recurrent.cuda()

lstm_loss = LSTM_loss(1,3,3,res,(H,W),a_cuda)
losses = []

# use ADAM optimizer
optimizer = optim.Adam(recurrent.parameters(), lr=3E-2, weight_decay=0.0005)

for i in [0]:
  labels, images = create_LSTMbatch(i, True)
  ntrain = images.shape[0]
  # for loop to get from different batches of read-in images
  for iepoch in range(nepoch_readbatch):
    num_its = int(ntrain/batchsize)
    ep_loss = np.zeros(num_its) 
    print(iepoch)
    for t in range(num_its):
        batchindices = np.random.choice(ntrain, batchsize, replace=False)

        # before the forward pass, clean the gradient buffers of all parameters
        optimizer.zero_grad()

        # forward pass
        this_batch = Variable(images[batchindices,...].float())
        if(a_cuda):
          this_batch = this_batch.cuda()
        yolo_out = yolo(this_batch)
        yolo_out = nms_thresh(yolo_out, 0.65, anchors, a_cuda, res)
        
        lstm_input = shape_for_LSTM(yolo_out, a_cuda)
        lstm_output = recurrent(lstm_input)
        lstm_output = lstm_output.view(batchsize-1, 5, H, W)

        # MSE loss
        label_indices = (labels[:, -1][..., None] == torch.tensor(batchindices)).any(-1).nonzero().squeeze()
        true_out = labels[label_indices,:]
        if(a_cuda):
          true_out = true_out.cuda()
        
        loss = lstm_loss(lstm_output, true_out)
        ep_loss[t] = loss.item()

        # backward pass
        loss.backward()

        # update parameters using SGD
        optimizer.step()

        # remove from memory
        this_batch = None
        true_out = None
        yolo_out = None
        lstm_input = None
        lstm_output = None
        gc.collect()
        if(a_cuda):
          torch.cuda.empty_cache()

    losses.append(np.mean(ep_loss))

In [ ]:
fig, ax = plt.subplots()
ax.plot(losses)
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.title('LSTM Loss on Waymo Subset')

## Full-Scale Training

In [ ]:
anchors = torch.load("anch.pt")
model_save_name = 'lstm_checkpoint.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
resume = True

base_model_name = '1_fixed_checkpoint.pt'
base_path = F"/content/gdrive/My Drive/{base_model_name}" 
check = torch.load(base_path)

tot_train = torch.load("train_idx.pt")  # number of overall training examples
tot_valid = torch.load("valid_idx.pt")
n_tot_train = len(tot_train)

nepoch_dataset = 20            # number of epochs through training set
batchsize = 16                 # minibatch size
a_cuda = True                  # whether or not to enable cuda gpu acceleration

res = 32                       # resolution of image breakdown
H = 1280//(2*res)                   # height of the grid over images
W = 1920//(2*res)                   # width of the grid over images
epoch = 0

network_spec = parse_cfg("yolo.cfg")
module_list = create_network(network_spec, 3)
yolo = Model(module_list).float()
yolo.load_state_dict(check['model_state_dict'])
recurrent = LSTM_module(W*H*50, 360, 2, W*H*5)

if(a_cuda):
  yolo = yolo.cuda()
  recurrent = recurrent.cuda()

lstm_loss = LSTM_loss(1,3,3,res,(H,W),a_cuda)
train_losses = []
valid_losses = []

# use ADAM optimizer
optimizer = optim.Adam(recurrent.parameters(), lr=3E-2, weight_decay=0.0005)

# Code for resume
if(resume):
  checkpoint = torch.load(path)
  recurrent.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']+1
  train_losses = checkpoint['loss']
  valid_losses = checkpoint['valid']

num_images = 200*n_tot_train
num_steps = num_images//batchsize

for iepoch in range(epoch, nepoch_dataset):
  ep_loss = np.zeros(num_steps)
  print("Epoch:" +str(iepoch))
  # for loop to train through different batches of read-in images
  for j in range(num_steps):
    print(str(j) + "/" + str(num_steps))
    index = int(np.random.choice(tot_train,1)[0])
    labels, images = create_LSTMbatch(index, True)
    ntrain = images.shape[0]
    batchindices = np.random.choice(ntrain, batchsize, replace=False)

    # before the forward pass, clean the gradient buffers of all parameters
    optimizer.zero_grad()

    # forward pass
    this_batch = Variable(images[batchindices,...].float())
    if(a_cuda):
      this_batch = this_batch.cuda()
    with(torch.no_grad()):
      yolo_out = yolo(this_batch)
        
    lstm_input = shape_for_LSTM(yolo_out, a_cuda)
    lstm_output = recurrent(lstm_input)
    lstm_output = lstm_output.view(batchsize-1, 5, H, W)

    # MSE loss
    label_indices = (labels[:, -1][..., None] == torch.tensor(batchindices)).any(-1).nonzero().squeeze()
    true_out = labels[label_indices,:]
    if(a_cuda):
      true_out = true_out.cuda()
        
    loss = lstm_loss(lstm_output, true_out)
    ep_loss[j] = loss.item()

    # backward pass
    loss.backward()

    # update parameters using SGD
    optimizer.step()

    # remove from memory
    this_batch = None
    true_out = None
    yolo_out = None
    lstm_input = None
    lstm_output = None
    gc.collect()
    if(a_cuda):
      torch.cuda.empty_cache()

  # Record training loss
  train_losses.append(np.mean(ep_loss))

  # Calculate and record validation loss using test set
  valid = np.zeros(len(tot_valid))
  for i in range(len(tot_valid)):
    index = int(tot_valid[i])
    labels, images = create_LSTMbatch(index, True)
    ntest = images.shape[0]
    batchindices = np.random.choice(ntest, batchsize, replace=False)
    this_batch = Variable(images[batchindices,...].float())
    if(a_cuda):
      this_batch = this_batch.cuda()
    yolo_out = yolo(this_batch)
    lstm_input = shape_for_LSTM(yolo_out, a_cuda)
    lstm_output = recurrent(lstm_input)
    lstm_output = lstm_output.view(batchsize-1, 5, H, W)

    label_indices = (labels[:, -1][..., None] == torch.tensor(batchindices)).any(-1).nonzero().squeeze()
    true_out = labels[label_indices,:]
    if(a_cuda):
      true_out = true_out.cuda()
    valid[i] = lstm_loss(lstm_output, true_out)

    this_batch = None
    true_out = None
    yolo_out = None
    lstm_input = None
    lstm_output = None
    gc.collect()
    if(a_cuda):
      torch.cuda.empty_cache()
  valid_losses.append(np.mean(valid))

  torch.save({
            'epoch': iepoch,
            'model_state_dict': recurrent.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_losses,
            'valid': valid_losses
            }, path)

Epoch:2
0/125
1/125
2/125
3/125
4/125
5/125
6/125
7/125
8/125
9/125
10/125
11/125
12/125
13/125
14/125
15/125
16/125
17/125
18/125
19/125
20/125
21/125
22/125
23/125
24/125
25/125
26/125
27/125
28/125
29/125
30/125
31/125
32/125
33/125
34/125
35/125
36/125
37/125
38/125
39/125
40/125
41/125
42/125
43/125
44/125
45/125
46/125
47/125
48/125
49/125
50/125
51/125
52/125
53/125
54/125
55/125
56/125
57/125
58/125
59/125
60/125
61/125
62/125
63/125
64/125
65/125
66/125
67/125
68/125
69/125
70/125
71/125
72/125
73/125
74/125
75/125
76/125
77/125
78/125
79/125
80/125
81/125
82/125
83/125
84/125
85/125
86/125
87/125
88/125
89/125
90/125
91/125
92/125
93/125
94/125
95/125
96/125
97/125
98/125
99/125
100/125
101/125
102/125
103/125
104/125
105/125
106/125
107/125
108/125
109/125
110/125
111/125
112/125
113/125
114/125
115/125
116/125
117/125
118/125
119/125
120/125
121/125
122/125
123/125
124/125
Epoch:3
0/125
1/125
2/125
3/125
4/125
5/125
6/125
7/125
8/125
9/125
10/125
11/125
12/125
13/125
14/125